In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AtmoBuilder import AtmoBuilder

import lsst.sims.photUtils.Sed as Sed
import lsst.sims.photUtils.Bandpass as Bandpass

In [2]:
WAVELENMIN = 300
WAVELENMAX = 1100
WAVELENSTEP = 0.1

STDPARAMETERS = [1.0,1.0,1.0,1.0,1.0,1.7]
STDAIRMASS = 1.2
STDAEROSOLNORMCOEFF = 0.1
STDAEROSOLNORMWAVELEN = 550.0
STDAEROSOLALPHA = STDPARAMETERS[5]

sedFiles = ["BC2003_0.571gyr_solar.sed0", "Sc_template_norm.sed0"]

In [3]:
ab = AtmoBuilder()

Found 16 MODTRAN files:
Pachon_MODTRAN.10.7sc
Pachon_MODTRAN.11.7sc
Pachon_MODTRAN.12.7sc
Pachon_MODTRAN.13.7sc
Pachon_MODTRAN.14.7sc
Pachon_MODTRAN.15.7sc
Pachon_MODTRAN.16.7sc
Pachon_MODTRAN.17.7sc
Pachon_MODTRAN.18.7sc
Pachon_MODTRAN.19.7sc
Pachon_MODTRAN.20.7sc
Pachon_MODTRAN.21.7sc
Pachon_MODTRAN.22.7sc
Pachon_MODTRAN.23.7sc
Pachon_MODTRAN.24.7sc
Pachon_MODTRAN.25.7sc
MODTRAN files have been read.

Read filter data from LSST software stack.
Filters: ['u', 'g', 'r', 'i', 'z', 'y4']
Read hardware data from LSST software stack.


In [4]:
def readSedAndRedshift(sedFile, redshiftRange=[0, 3.0], redshiftStep=0.001, addDust=True):
    
    wavelength_angstroms, fnu = np.loadtxt(sedFile, unpack=True)
    wavelength_nm = wavelength_angstroms*0.1
    baseSed = Sed(wavelen=wavelength_nm, fnu=fnu)
        
    seds = {}
    sedkeylist = []
    redshifts = np.arange(redshiftRange[0], redshiftRange[1] + redshiftStep, redshiftStep)
    for z in redshifts:
        sedname = "%s_%.3f" % (sedFile, z)
        wavelen, flambda = baseSed.redshiftSED(z, wavelen=baseSed.wavelen, flambda=baseSed.flambda)
        seds[sedname] = Sed(wavelen=wavelen, flambda=flambda)
        sedkeylist.append(sedname)
            
    for sed in sedkeylist:
        seds[sed].synchronizeSED(wavelen_min=WAVELENMIN, wavelen_max=WAVELENMAX, wavelen_step=WAVELENSTEP)
    
    if addDust:
        ax, bx = seds[sedkeylist[0]].setupCCMab()
        for sed in sedkeylist:
            seds[sed].addCCMDust(ax, bx, A_v=0.02)
        
    return seds, sedkeylist, redshifts

def buildPanel(seds, sedkeylist, redshifts, airmass=1.0, parameters=[1.0,1.0,1.0,1.0,1.0,1.7]):
    atmo = ab.buildAtmo(parameters, airmass)
    throughput = ab.combineThroughputs(atmo)
    mags = ab.mags(throughput, seds=seds, sedkeylist=sedkeylist)
    
    color_dict = {}
    colors = ["u-g", "g-r", "r-i", "i-z", "z-y4"]
    for color in colors:
        filters = color.split("-")
        color_dict[color] = mags[filters[0]] - mags[filters[1]]

    panel = np.zeros(len(sedkeylist), 
            dtype={"names":["z", "u-g", "g-r", "r-i", "i-z", "z-y4"], 
                    "formats":["float64","float64","float64","float64","float64","float64"]})
    
    panel["z"] = redshifts
    for color in colors:
        panel[color] = color_dict[color]
        
    df = pd.DataFrame(panel, columns=["u-g", "g-r", "r-i", "i-z", "z-y4"], index=panel["z"])
    df.index.name = "z"
    return df
    
def buildTable(sedFile, airmasses=[1.0,1.5,2.0], parameters=[1.0,1.0,1.0,1.0,1.0,1.7], redshiftRange=[0, 3.0], redshiftStep=0.001, addDust=True):
    
    seds, sedkeylist, redshifts = readSedAndRedshift(sedFile, redshiftRange=redshiftRange, redshiftStep=redshiftStep, addDust=addDust)

    panels = []
    for airmass in airmasses:
        panels.append(buildPanel(seds, sedkeylist, redshifts, airmass=airmass, parameters=parameters))

    return pd.concat(panels, keys=airmasses)

In [5]:
BC = buildTable(sedFiles[0])

In [6]:
BC

u-g       g-r       r-i       i-z      z-y4
    z                                                      
1.0 0.000  0.637291 -0.322809 -0.303299 -0.218863 -0.170111
    0.001  0.642287 -0.322240 -0.303034 -0.219490 -0.169298
    0.002  0.649003 -0.321965 -0.302790 -0.219331 -0.169148
    0.003  0.654477 -0.321435 -0.302287 -0.218737 -0.170333
    0.004  0.659051 -0.321000 -0.302446 -0.219103 -0.169618
    0.005  0.664973 -0.319351 -0.302375 -0.220046 -0.169403
    0.006  0.668486 -0.317477 -0.302980 -0.218831 -0.170440
    0.007  0.671619 -0.316867 -0.301925 -0.219342 -0.169655
    0.008  0.677100 -0.315718 -0.302081 -0.218814 -0.169659
    0.009  0.682132 -0.314205 -0.302633 -0.218561 -0.170041
    0.010  0.686343 -0.314202 -0.302322 -0.218903 -0.169533
    0.011  0.692166 -0.314022 -0.301246 -0.219319 -0.169511
    0.012  0.696662 -0.312557 -0.301827 -0.218788 -0.169717
    0.013  0.700997 -0.310927 -0.301676 -0.219503 -0.169204
    0.014  0.707372 -0.309783 -0.301881 -0.218768 -0.169362
    0.015  0.710932 -0.309132 -0.301140 -0.219510 -0.169208
    0.016  0.714628 -0.309092 -0.301008 -0.219884 -0.168591
    0.017  0.722393 -0.308664 -0.300066 -0.219594 -0.169704
    0.018  0.727448 -0.307318 -0.300939 -0.219388 -0.169450
    0.019  0.728439 -0.305545 -0.301149 -0.220304 -0.168975
    0.020  0.733933 -0.304425 -0.300953 -0.219719 -0.169180
    0.021  0.739468 -0.303484 -0.300216 -0.220087 -0.168783
    0.022  0.741093 -0.302528 -0.299826 -0.220655 -0.168198
    0.023  0.745142 -0.301575 -0.300375 -0.219416 -0.168797
    0.024  0.750535 -0.300731 -0.299722 -0.220406 -0.168580
    0.025  0.753066 -0.299497 -0.300019 -0.219783 -0.168936
    0.026  0.755796 -0.297741 -0.299717 -0.220548 -0.168692
    0.027  0.759821 -0.296246 -0.299662 -0.220412 -0.168773
    0.028  0.762782 -0.295142 -0.299854 -0.219926 -0.168679
    0.029  0.765870 -0.293852 -0.299491 -0.220862 -0.168401
...             ...       ...       ...       ...       ...
2.0 2.971  0.886155  2.991167  1.145040  0.036279 -0.288882
    2.972  0.883612  2.991214  1.147132  0.037477 -0.289399
    2.973  0.879731  2.992692  1.149168  0.038711 -0.289901
    2.974  0.876352  2.993458  1.151364  0.039913 -0.290338
    2.975  0.876547  2.992601  1.153551  0.041120 -0.290783
    2.976  0.871640  2.993347  1.155582  0.042295 -0.291149
    2.977  0.867236  2.994940  1.157807  0.043495 -0.291482
    2.978  0.864568  2.995048  1.160043  0.044620 -0.291785
    2.979  0.863076  2.994154  1.162094  0.045724 -0.292040
    2.980  0.860810  2.995206  1.164392  0.046917 -0.292270
    2.981  0.855556  2.997039  1.166564  0.047985 -0.292497
    2.982  0.852471  2.997110  1.168638  0.049165 -0.292676
    2.983  0.851026  2.995800  1.171044  0.050256 -0.292865
    2.984  0.847267  2.997176  1.173157  0.051321 -0.292958
    2.985  0.843966  2.999012  1.175327  0.052436 -0.293137
    2.986  0.841600  2.998486  1.177735  0.053499 -0.293158
    2.987  0.838658  2.998070  1.179838  0.054552 -0.293360
    2.988  0.834898  2.999278  1.182045  0.055624 -0.293364
    2.989  0.831037  3.000488  1.184400  0.056695 -0.293564
    2.990  0.829648  3.000094  1.186563  0.057729 -0.293584
    2.991  0.827586  3.000267  1.188692  0.058800 -0.293809
    2.992  0.822385  3.001418  1.191090  0.059815 -0.293844
    2.993  0.818883  3.002392  1.193266  0.060878 -0.294085
    2.994  0.817251  3.001684  1.195272  0.061965 -0.294169
    2.995  0.814358  3.002168  1.197699  0.062971 -0.294435
    2.996  0.811606  3.003642  1.199867  0.064036 -0.294515
    2.997  0.806624  3.004520  1.201879  0.065145 -0.294847
    2.998  0.805343  3.003097  1.204218  0.066183 -0.294968
    2.999  0.801707  3.004187  1.206385  0.067292 -0.295331
    3.000  0.797756  3.005673  1.208486  0.068336 -0.295496

[9003 rows x 5 columns]

In [7]:
Sc = buildTable(sedFiles[1])

In [8]:
Sc

u-g       g-r       r-i       i-z      z-y4
    z                                                      
1.0 0.000  0.774773  0.067711 -0.117101 -0.051993 -0.022854
    0.001  0.776380  0.069175 -0.117428 -0.051831 -0.022345
    0.002  0.777932  0.070668 -0.117753 -0.051668 -0.021828
    0.003  0.779419  0.072191 -0.118081 -0.051504 -0.021324
    0.004  0.780829  0.073737 -0.118398 -0.051334 -0.020802
    0.005  0.782172  0.075295 -0.118689 -0.051184 -0.020256
    0.006  0.783449  0.076864 -0.118945 -0.051084 -0.019703
    0.007  0.784711  0.078438 -0.119175 -0.051035 -0.019130
    0.008  0.785951  0.080009 -0.119363 -0.051061 -0.018534
    0.009  0.787195  0.081585 -0.119522 -0.051166 -0.017946
    0.010  0.788472  0.083145 -0.119622 -0.051354 -0.017395
    0.011  0.789757  0.084717 -0.119676 -0.051604 -0.016860
    0.012  0.791038  0.086284 -0.119678 -0.051934 -0.016306
    0.013  0.792310  0.087854 -0.119651 -0.052290 -0.015817
    0.014  0.793571  0.089431 -0.119601 -0.052667 -0.015352
    0.015  0.794743  0.090996 -0.119535 -0.053078 -0.014845
    0.016  0.795910  0.092583 -0.119469 -0.053477 -0.014344
    0.017  0.796928  0.094206 -0.119412 -0.053882 -0.013852
    0.018  0.797837  0.095852 -0.119358 -0.054310 -0.013315
    0.019  0.798664  0.097483 -0.119251 -0.054760 -0.012722
    0.020  0.799388  0.099111 -0.119121 -0.055222 -0.012133
    0.021  0.799966  0.100796 -0.118993 -0.055660 -0.011571
    0.022  0.800504  0.102484 -0.118848 -0.056105 -0.011022
    0.023  0.800933  0.104178 -0.118695 -0.056586 -0.010487
    0.024  0.801358  0.105873 -0.118486 -0.057126 -0.010008
    0.025  0.801751  0.107547 -0.118219 -0.057712 -0.009573
    0.026  0.802071  0.109237 -0.117936 -0.058324 -0.009144
    0.027  0.802451  0.110935 -0.117641 -0.058936 -0.008706
    0.028  0.802905  0.112624 -0.117337 -0.059527 -0.008267
    0.029  0.803186  0.114344 -0.117038 -0.060133 -0.007803
...             ...       ...       ...       ...       ...
2.0 2.971  5.024746  0.044228 -0.271570 -0.232461 -0.253520
    2.972  5.054021  0.045276 -0.271614 -0.232145 -0.254011
    2.973  5.084107  0.046385 -0.271672 -0.231828 -0.254507
    2.974  5.115053  0.047423 -0.271691 -0.231504 -0.254963
    2.975  5.115250  0.048285 -0.271719 -0.231200 -0.255417
    2.976  5.139439  0.049350 -0.271779 -0.230875 -0.255843
    2.977  5.168122  0.050440 -0.271815 -0.230559 -0.256261
    2.978  5.197601  0.051523 -0.271835 -0.230251 -0.256647
    2.979  5.227877  0.052642 -0.271898 -0.229939 -0.257032
    2.980  5.227320  0.053468 -0.271920 -0.229633 -0.257369
    2.981  5.251182  0.054552 -0.271962 -0.229323 -0.257724
    2.982  5.278637  0.055684 -0.272017 -0.229017 -0.258018
    2.983  5.306794  0.056781 -0.272020 -0.228714 -0.258343
    2.984  5.335702  0.057909 -0.272080 -0.228407 -0.258612
    2.985  5.334371  0.058758 -0.272124 -0.228099 -0.258911
    2.986  5.358157  0.059834 -0.272129 -0.227805 -0.259154
    2.987  5.384700  0.060981 -0.272193 -0.227500 -0.259413
    2.988  5.411902  0.062125 -0.272228 -0.227211 -0.259649
    2.989  5.439794  0.063227 -0.272241 -0.226918 -0.259884
    2.990  5.437807  0.064068 -0.272284 -0.226624 -0.260107
    2.991  5.460869  0.065218 -0.272326 -0.226356 -0.260337
    2.992  5.485833  0.066327 -0.272326 -0.226068 -0.260551
    2.993  5.511375  0.067478 -0.272338 -0.225806 -0.260793
    2.994  5.537526  0.068648 -0.272381 -0.225541 -0.261003
    2.995  5.534953  0.069436 -0.272366 -0.225265 -0.261250
    2.996  5.558168  0.070586 -0.272367 -0.225025 -0.261461
    2.997  5.582565  0.071762 -0.272395 -0.224758 -0.261707
    2.998  5.607519  0.072893 -0.272359 -0.224522 -0.261921
    2.999  5.633055  0.074052 -0.272362 -0.224282 -0.262155
    3.000  5.629981  0.074871 -0.272372 -0.224026 -0.262384

[9003 rows x 5 columns]

In [9]:
### SANITY CHECK ###
# As a sanity check, I’d expect that the colors are very similar for the first
# galaxy (BC) at z=0.38 and the second one (Sc) at z=3.55. 
BC_test = buildTable(sedFiles[0], airmasses=[1.0], redshiftRange=[0,3.6])
Sc_test = buildTable(sedFiles[1], airmasses=[1.0], redshiftRange=[0,3.6])

BC_test.loc[1.0, 0.380]
Sc_test.loc[1.0, 3.550]

Exception: This SED has no flux within this bandpass.